# **Práctica modelo no supervisado**

## Análisis de segmentación con K-Means sobre datos del Censo

En este ejercicio aplicaremos un modelo **de K-Means** para identificar **grupos o perfiles de personas** con características similares.

El **K-Means** es un algoritmo de **aprendizaje no supervisado** que se utiliza para realizar **análisis de clúster o segmentación**.  

Su objetivo es **agrupar observaciones similares** en función de sus características, **sin conocer etiquetas o clases previas**.

---

**IMPORTANTE:**

A diferencia de la regresión logística, donde buscábamos **predecir si una persona trabaja o no**, en este caso **no hay una variable objetivo**.  

El propósito es **explorar patrones naturales** en los datos y analizar si los clústeres formados reflejan perfiles sociodemográficos coherentes.

![K-Means ejemplo](https://i.imgur.com/aDTF2cZ.png)

### Métricas de calidad del modelo

- **Inercia (SSE):** mide la suma de distancias internas en cada clúster.  
  Cuanto menor, más compactos son los grupos.  

- **Coeficiente Silhouette:** mide la separación entre clústeres.  

  Valores cercanos a **1** indican buena separación; valores cercanos a **0** o negativos, solapamiento entre grupos.




### **I. Configuración del entorno (SparkSession)**


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.functions import vector_to_array
import matplotlib.pyplot as plt

### **II. Carga y exploración inicial de datos**


In [ ]:
!apt-get update -y
!apt-get install -y openjdk-17-jdk
!pip install -q "pyspark>=3.6,<3.7"
!pip install -q "pyspark>=3.6,<3.7"

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

os.environ.pop("PYSPARK_SUBMIT_ARGS", None)
os.environ.pop("SPARK_HOME", None)


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [3]:
spark = (SparkSession.builder
         .appName("KMeans-demo")
         .getOrCreate())

In [4]:
sdf = spark.read.csv('personas_ext_26_02.csv', header=True, inferSchema=True)


sdf.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID_CENSO: double (nullable = true)
 |-- DIRECCION_ID: string (nullable = true)
 |-- DEPARTAMENTO: integer (nullable = true)
 |-- LOCALIDAD: integer (nullable = true)
 |-- VIVID: string (nullable = true)
 |-- HOGID: string (nullable = true)
 |-- PERID: string (nullable = true)
 |-- REGION_4: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- MUNICIPIO_PAIS: string (nullable = true)
 |-- TIPO_MUNICIPIO_PAIS: string (nullable = true)
 |-- FUENTE_EXT: integer (nullable = true)
 |-- SIT_CALLE: integer (nullable = true)
 |-- CUESTIONARIO_COMPLETO: integer (nullable = true)
 |-- CUESTIONARIO_BASICO: integer (nullable = true)
 |-- RRAA: integer (nullable = true)
 |-- UNIVERSO: integer (nullable = true)
 |-- VIVVO00: integer (nullable = true)
 |-- PERPH02: integer (nullable = true)
 |-- PERNA01: integer (nullable = true)
 |-- PERNA01_TRAMO: string (nullable = true)
 |-- PERPA01: integer (nullable = true)
 |-- PERPA02: integer (nulla

In [5]:
sdf.show(5)

+---+--------+------------+------------+---------+-----+-----+-----+--------+----+--------------+-------------------+----------+---------+---------------------+-------------------+----+--------+-------+-------+-------+-------------+-------+-------+---------+-------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---------+---------+-------+-------+---------+-------+---------+---------+-------+---------+---------+-------+-------+-------+-------+-------+---------+---------+-------+---------+-------+-------+---------+-----------+---------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------+-------+---------+---------+-------+---------+---------+-------+---------+-------+--------+--------+------------------------+-----------+----------+---------------+----------+---------------+------------+-----------------+-------+------------+
|_c

### **III. Análisis descriptivo y preprocesamiento**

Usaremos las siguientes variables:
- `edad`: edad en años (variable continua)  
- `sexo`: variable binaria (0 = mujer, 1 = hombre)  
- `educacion`: nivel educativo codificado de menor a mayor  
- `educacion_orden`: orden o jerarquía del nivel educativo  

La variable `trabaja` se mantendrá únicamente para un **análisis posterior (post-hoc)**, es decir, para observar cómo se distribuye el empleo dentro de los clústeres obtenidos, pero **no participará en el entrenamiento del modelo**.

In [6]:
sdf_small = sdf.select(["PERAL01", "PERNA01", "PERPH02", "PERED03"])

In [7]:
sdf_small.show(3)

+-------+-------+-------+-------+
|PERAL01|PERNA01|PERPH02|PERED03|
+-------+-------+-------+-------+
|   8888|     74|      2|   8888|
|   8888|     37|      1|   8888|
|   8888|     39|      2|   8888|
+-------+-------+-------+-------+
only showing top 3 rows



In [8]:
sdf_clean = (sdf_small
    .withColumnRenamed("PERAL01", "trabaja")
    .withColumnRenamed("PERPH02", "sexo")
    .withColumnRenamed("PERNA01", "edad")
    .withColumnRenamed("PERED03", "educacion")
)

In [9]:
sdf_clean = sdf_clean.withColumn(
    "edad",
    F.when((F.col("edad") >= 0) & (F.col("edad") <= 100), F.col("edad")).otherwise(None)
)

In [10]:
sdf_clean = sdf_clean.withColumn(
    "trabaja",
    F.when(F.col("trabaja") == 1, 1)
     .when(F.col("trabaja") == 2, 0)
     .otherwise(None)
)

In [11]:
sdf_clean = sdf_clean.withColumn(
    "sexo",
    F.when(F.col("sexo").isin(1, 2), F.col("sexo")).otherwise(None)
)

In [12]:
sdf_clean = sdf_clean.withColumn(
    "educacion",
    F.when((F.col("educacion") >= 1) & (F.col("educacion") <= 20), F.col("educacion")).otherwise(None)
)

In [13]:
sdf_clean = sdf_clean.withColumn(
    "educacion_orden",
    F.when(F.col("educacion") == 1,  1) \
     .when(F.col("educacion") == 2,  2) \
     .when(F.col("educacion") == 3,  3) \
     .when(F.col("educacion") == 15, 4) \
     .when(F.col("educacion") == 13, 5) \
     .when(F.col("educacion") == 14, 6) \
     .when(F.col("educacion") == 9,  7) \
     .when(F.col("educacion") == 10, 8) \
     .when(F.col("educacion") == 11, 9) \
     .when(F.col("educacion") == 12, 10) \
     .otherwise(None)
)

In [14]:
sdf_clean = sdf_clean.dropna(subset=["trabaja","edad","sexo","educacion_orden"])

In [15]:
sdf_clean.show(3)

+-------+----+----+---------+---------------+
|trabaja|edad|sexo|educacion|educacion_orden|
+-------+----+----+---------+---------------+
|      1|  37|   1|        9|              7|
|      1|  30|   1|        9|              7|
|      0|  12|   1|       13|              5|
+-------+----+----+---------+---------------+
only showing top 3 rows



In [16]:
sdf_clean.describe().show()

+-------+------------------+------------------+-------------------+------------------+-----------------+
|summary|           trabaja|              edad|               sexo|         educacion|  educacion_orden|
+-------+------------------+------------------+-------------------+------------------+-----------------+
|  count|            431315|            431315|             431315|            431315|           431315|
|   mean|0.3053754216755735| 22.22648180564089| 1.5490604314711987|11.858592907735646|6.482603201836245|
| stddev|0.4605667870292752|10.650285721995893|0.49758782954121217|2.7325263868756653|1.971456295785069|
|    min|                 0|                12|                  1|                 2|                2|
|    max|                 1|                78|                  2|                15|               10|
+-------+------------------+------------------+-------------------+------------------+-----------------+



1) Seleccionar variables (excluyendo trabaja)


In [17]:
feature_cols = ["edad", "sexo", "educacion", "educacion_orden"]


2) Ensamblar y escalar (estandarización)

In [18]:
assembler = VectorAssembler(
    inputCols=["edad", "sexo", "educacion", "educacion_orden"],
    outputCol="features_raw"
)

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=True,
    withStd=True
)

prep_pipeline = Pipeline(stages=[assembler, scaler]).fit(sdf_clean)

df_prep = prep_pipeline.transform(sdf_clean)

3) BUSCAR EL MEJOR NÚMERO DE CLUSTERS (k)

In [19]:
evaluator = ClusteringEvaluator(
    featuresCol="features",
    predictionCol="prediction",
    metricName="silhouette"
)

best_k = None
best_sil = -1.0
best_sse = float("inf")
best_model = None

print("=== Evaluación K-Means: Silhouette + SSE ===")

for k in range(2, 7):
    kmeans = KMeans(featuresCol="features", k=k, seed=42)
    model = kmeans.fit(df_prep)
    preds = model.transform(df_prep)
    sil = evaluator.evaluate(preds)
    sse = model.summary.trainingCost
    print(f"k = {k} → Silhouette = {sil:.4f} | SSE = {sse:.2f}")
    if sil > best_sil:
        best_sil = sil
        best_sse = sse
        best_k = k
        best_model = model

print(f"\n✅ Mejor k según Silhouette = {best_k}")
print(f"   Silhouette = {best_sil:.4f}")
print(f"   SSE        = {best_sse:.2f}")


=== Evaluación K-Means: Silhouette + SSE ===
k = 2 → Silhouette = 0.4475 | SSE = 1285432.14
k = 3 → Silhouette = 0.4552 | SSE = 990597.24
k = 4 → Silhouette = 0.5351 | SSE = 804533.33
k = 5 → Silhouette = 0.6351 | SSE = 477695.50
k = 6 → Silhouette = 0.7074 | SSE = 380275.06

✅ Mejor k según Silhouette = 6
   Silhouette = 0.7074
   SSE        = 380275.06


4) ENTRENAR MODELO FINAL Y ASIGNAR CLUSTERS



In [20]:
df_clusters = best_model.transform(df_prep).withColumnRenamed("prediction", "cluster")

print("Tamaño de cada clúster:")
df_clusters.groupBy("cluster").count().orderBy("cluster").show()

Tamaño de cada clúster:
+-------+------+
|cluster| count|
+-------+------+
|      0|115273|
|      1| 16672|
|      2| 66597|
|      3|108903|
|      4|  6421|
|      5|117449|
+-------+------+



En el análisis de **perfiles latentes (diferente a clases latentes)**, se evita y excluye los clústeres con un tamaño menor a 3%, para asegurar la representatividad de cada perfil identificado (Spurk et al., 2020).


**Referencias** 

Spurk, D., Hirschi, A., Wang, M., Valero, D., & Kauffeld, S. (2020). Latent profile analysis: A review and “how to” guide of its application within vocational behavior research. Journal of Vocational Behavior, 120, 103445. https://doi.org/10.1016/j.jvb.2020.103445

5) PERFILADO DE CLUSTERS

Medias de las variables numéricas por clúster

In [21]:
perfil = (
    df_clusters.groupBy("cluster")
    .agg(
        F.mean("edad").alias("edad_media"),
        F.mean("sexo").alias("sexo_media"),
        F.mean("educacion_orden").alias("edu_orden_media")
    )
    .orderBy("cluster")
)
perfil.show(truncate=False)



+-------+------------------+------------------+------------------+
|cluster|edad_media        |sexo_media        |edu_orden_media   |
+-------+------------------+------------------+------------------+
|0      |16.582920545140666|1.0               |5.4042924188665165|
|1      |14.575815738963533|1.4291026871401151|2.2094529750479848|
|2      |28.462288091055154|1.0               |8.518521855338829 |
|3      |29.49573473641681 |2.0               |8.42707730732854  |
|4      |50.64522660021804 |1.5158075066189067|2.1138451954524218|
|5      |17.021634922391847|2.0               |5.428943626595373 |
+-------+------------------+------------------+------------------+



6) Porcentaje de personas que trabajan en cada clúster (post-hoc)


In [22]:
trabaja_por_cluster = (
    df_clusters.groupBy("cluster")
    .agg(F.mean(F.col("trabaja").cast("double")).alias("porc_trabaja"))
    .orderBy("cluster")
)
print("Porcentaje que trabaja por clúster:")
trabaja_por_cluster.show(truncate=False)

Porcentaje que trabaja por clúster:
+-------+-------------------+
|cluster|porc_trabaja       |
+-------+-------------------+
|0      |0.10541063388651288|
|1      |0.04360604606525912|
|2      |0.5828040302115711 |
|3      |0.5848231912803137 |
|4      |0.5259305404142657 |
|5      |0.11031170976338667|
+-------+-------------------+



In [23]:
spark.stop()

### Material complementario:
[Clustering con Python](https://cienciadedatos.net/documentos/py20-clustering-con-python)


[Introducción a k-Means Clustering con scikit-learn en Python](https://www.datacamp.com/es/tutorial/k-means-clustering-python)


[Elbow Method for optimal value of k in KMeans](https://www.geeksforgeeks.org/machine-learning/elbow-method-for-optimal-value-of-k-in-kmeans/)

[Understanding Silhouette Score in Clustering](https://farshadabdulazeez.medium.com/understanding-silhouette-score-in-clustering-8aedc06ce9c4)



